In [18]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [19]:
import os
main_dir ='New Masks Dataset'
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non Mask')

In [20]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_nomask_dir)
print(train_nomask_names[:10])

['0003.jpg', '0006.jpg', '0018.jpg', '0019.jpg', '0022.jpg', '0027.jpg', '0029.jpg', '0037.jpg', '0040.jpg', '0042.jpg']
['0.jpg', '1.jpg', '10.jpg', '100.jpg', '101.jpg', '102.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg']


In [21]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen =ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size =(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary')
test_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size =(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary')
valid_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size =(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [22]:
print(train_generator.class_indices)

{'Mask': 0, 'Non Mask': 1}


In [23]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation = 'relu',input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 75, 75, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 37, 37, 64)       

In [24]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\SUDHANSHU\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
history = model.fit(train_generator,
                    epochs = 30,
                    validation_data = valid_generator)

Epoch 1/30
19/19 [==============================] - 17s 849ms/step - loss: 3.9979 - accuracy: 0.5283 - val_loss: 0.6931 - val_accuracy: 0.5017
Epoch 2/30
19/19 [==============================] - 15s 802ms/step - loss: 0.6896 - accuracy: 0.5500 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 3/30
19/19 [==============================] - 15s 800ms/step - loss: 0.6397 - accuracy: 0.6000 - val_loss: 0.6251 - val_accuracy: 0.7617
Epoch 4/30
19/19 [==============================] - 15s 804ms/step - loss: 0.4601 - accuracy: 0.7933 - val_loss: 0.4623 - val_accuracy: 0.8083
Epoch 5/30
19/19 [==============================] - 15s 797ms/step - loss: 0.3405 - accuracy: 0.8717 - val_loss: 0.3812 - val_accuracy: 0.8450
Epoch 6/30
19/19 [==============================] - 19s 1s/step - loss: 0.2795 - accuracy: 0.8967 - val_loss: 0.3598 - val_accuracy: 0.8517
Epoch 7/30
19/19 [==============================] - 15s 806ms/step - loss: 0.2615 - accuracy: 0.9083 - val_loss: 0.3282 - val_accuracy: 0.8683
Ep

In [28]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss:{}'.format(test_acc,test_loss))

19/19 [==============================] - 6s 312ms/step - loss: 0.1837 - accuracy: 0.9433
test acc :0.9433333277702332 test loss:0.18367449939250946


In [29]:
model.save('held_model.h5')